# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,22.44,86,100,6.17,US,2025-03-20 12:13:15
1,1,bethel,41.3712,-73.4140,5.18,92,99,0.89,US,2025-03-20 12:13:16
2,2,bodaybo,57.8506,114.1933,-11.28,93,100,0.50,RU,2025-03-20 12:13:17
3,3,waitangi,-43.9535,-176.5597,13.79,90,93,0.89,NZ,2025-03-20 12:13:18
4,4,wickenburg,33.9686,-112.7296,8.33,32,63,0.89,US,2025-03-20 12:13:19


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
# Create an interactive map with city locations and humidity as the size of the points
city_map = city_data_df.hvplot.points(
    'Lng', 'Lat',  # Longitude and Latitude coordinates of the cities
    geo=True,  # geographic plotting
    size='Humidity',  # Size of points is based on humidity
    color='Humidity',  # Color of points is based on humidity
    cmap='Blues',  # Use the 'Blues' color map
    hover_cols=['City', 'Humidity'],  # Show hover info including the City and Humidity
    title="City Locations with Humidity",
    width=800,
    height=600,
    tiles = 'OSM',
    frame_width = 800,
    frame_height=600
)

# Display the map
city_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
# Filter the city data for ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 17) & 
    (city_data_df['Max Temp'] <= 27) & 
    (city_data_df['Wind Speed'] < 4) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop rows with null values
ideal_weather_df.dropna(inplace=True)

# Display the filtered DataFrame
ideal_weather_df.head()


C:\Users\arkha\AppData\Local\Temp\ipykernel_5136\3432539571.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
99,99,colonia,50.9333,6.9500,19.41,40,0,2.06,DE,2025-03-20 12:15:15
101,101,vallenar,-28.5708,-70.7581,18.37,64,0,0.52,CL,2025-03-20 12:15:18
147,147,buka,40.8108,69.1986,26.85,11,0,2.63,UZ,2025-03-20 12:16:14
167,167,kerman,30.2832,57.0788,22.40,9,0,1.45,IR,2025-03-20 12:16:38
246,246,shahecheng,36.9383,114.5058,17.65,11,0,3.26,CN,2025-03-20 12:18:22


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Create the hotel_df DataFrame, keeping relevant columns
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Drop any rows with null values in the hotel_df DataFrame
hotel_df.dropna(inplace=True)

# Display sample data
hotel_df.head()


C:\Users\arkha\AppData\Local\Temp\ipykernel_5136\2998121975.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(inplace=True)


,City,Country,Lat,Lng,Humidity
99,colonia,DE,50.9333,6.9500,40
101,vallenar,CL,-28.5708,-70.7581,64
147,buka,UZ,40.8108,69.1986,11
167,kerman,IR,30.2832,57.0788,9
246,shahecheng,CN,36.9383,114.5058,11


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Ensure that hotel_df has the correct types and is a deep copy
hotel_df = hotel_df.copy()  # Avoid SettingWithCopyWarning
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].astype(str)  # Convert 'Hotel Name' column to str

# Define the radius for hotel search (10,000 meters)
radius = 10000

# Define parameters for Geoapify API search
params = {
    'categories': 'accommodation.hotel',  # Specify we are looking for hotels
    'limit': 1,  # Only return the first result
    'apiKey': geoapify_key,  
}

# Print a message to follow up the hotel search
print("Starting hotel search...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params['filter'] = f"circle:{longitude},{latitude},{radius}"  # Circle filter based on coordinates
    params['bias'] = f"proximity:{longitude},{latitude}"  # Bias the results towards the current city

    # Set the base URL for Geoapify API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Try to grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the updated hotel_df with the hotel names
hotel_df


Starting hotel search...
colonia - nearest hotel: Wasserturm Hotel Cologne
vallenar - nearest hotel: Hotel del Marqués
buka - nearest hotel: No hotel found
kerman - nearest hotel: حافظ
shahecheng - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
colonia menno - nearest hotel: Hotel Mora
milot - nearest hotel: Merci Patron Hotel
pyin oo lwin - nearest hotel: Hotel Pan Thitsar
ribas do rio pardo - nearest hotel: Hotel São Francisco
turpan - nearest hotel: Hua Sheng Hotel
kailua-kona - nearest hotel: PACIFIC 19 Kona
faaa - nearest hotel: Ancien Hôtel Hilton
rio brilhante - nearest hotel: Gold Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
99,colonia,DE,50.9333,6.9500,40,Wasserturm Hotel Cologne
101,vallenar,CL,-28.5708,-70.7581,64,Hotel del Marqués
147,buka,UZ,40.8108,69.1986,11,No hotel found
167,kerman,IR,30.2832,57.0788,9,حافظ
246,shahecheng,CN,36.9383,114.5058,11,No hotel found
264,holualoa,US,19.6228,-155.9522,85,Kona Hotel
289,colonia menno,PY,-22.3667,-59.8167,60,Hotel Mora
333,milot,HT,19.6167,-72.2167,70,Merci Patron Hotel
335,pyin oo lwin,MM,22.0333,96.4667,27,Hotel Pan Thitsar
429,ribas do rio pardo,BR,-20.4431,-53.7592,61,Hotel São Francisco


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

import hvplot.pandas  # for interactive plotting
import geopandas as gpd

# Configure the plot to display the city locations on a world map, with the size of the points based on humidity
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat',  # Longitude and Latitude for the coordinates of the cities
    geo=True,  # Enable geographic plotting to show on world map
    size='Humidity',  # Size of points is based on humidity
    color='Humidity',  # Color of points is based on humidity
    cmap='Blues',  # Color map
    hover_cols=['City', 'Hotel Name', 'Country'],  # Show hover info for city, hotel name, and country
    title="City Locations with Hotel Info",
    width=800,
    height=600,
    tiles='OSM',  # Set tile layer (Open Street Map in this case)
    frame_width=800,
    frame_height=600
)

# Display the map
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)